In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import os
import csv

# for progress bars
from tqdm import tqdm, tqdm_pandas
tqdm_pandas(tqdm())

# API Key
from config import api_key

#ZillowWrapper
from pyzillow.pyzillow import ZillowWrapper, GetDeepSearchResults

0it [00:00, ?it/s]


In [2]:
#Read csv
address_zip_df = pd.read_csv('ChicagoAddress.csv').dropna()
address_zip_df.head()

,Address,Zip
0,5 14th,60605
1,8 9th,60605
2,234 Ashland,60605
3,1733 Congress,60605
4,1753 Congress,60605


In [3]:
test_data = address_zip_df

In [4]:
test_data.shape

(21241, 2)

In [5]:
address_zip_df.shape

(21241, 2)

In [6]:
address = '233 E 13th St 2207'
zipcode = '60605'

zillow_data = ZillowWrapper(api_key)
response = zillow_data.get_deep_search_results(address, zipcode)
result = GetDeepSearchResults(response)
print(result.zestimate_valuation_range_high) #no
print(result.zestimate_valuationRange_low)#no
print(result.zestimate_value_change) #yes last 30 days
print(result.home_detail_link)#no
print(result.property_size) #yes
print(result.home_size) #no

563557
499151
4840
https://www.zillow.com/homedetails/233-E-13th-St-APT-2207-Chicago-IL-60605/70475126_zpid/
78844
1386


In [7]:
bad_address = "234 Ashland"
bad_zip = "60605"

In [8]:
good_address = "125 E 13th St 1001"
good_zip = "60605"

In [9]:
zillow_data = ZillowWrapper(api_key)

def get_zillow_info(address,zipcode):
    try:
        response = zillow_data.get_deep_search_results(address, zipcode)
        result = GetDeepSearchResults(response)
        
        # get zillow_id value
        try:
            zillow_id = result.zillow_id
        except AttributeError: 
            zillow_id = None
        
        # get tax value
        try:
            tax = result.tax_value
        except AttributeError: 
            tax = None
            
        # get latitude
        try:
            tax_year = result.tax_year
        except AttributeError:
            tax_year = None
            
        # 4home type 
        try:    
            home_type = result.home_type
        except AttributeError:
            home_type = None
            
        # 5 property size
        try:    
            property_size = result.property_size
        except AttributeError:
            property_size = None
        
        # 6 bathrooms
        try:    
            bathrooms = result.bathrooms
        except AttributeError:
            bathrooms = None
        
        # 7 bedrooms
        try:    
            bedrooms = result.bedrooms
        except AttributeError:
            bedrooms = None
            
        # 8last sold date
        try:    
            last_sold_date = result.last_sold_date
        except AttributeError:
            last_sold_date = None
        
        # 9last sold price
        try:    
            last_sold_price = result.last_sold_price
        except AttributeError:
            last_sold_price = None
            
       # 10zestimate amount
        try:    
            zestimate_amount = result.zestimate_amount
        except AttributeError:
            zestimate_amount = None
        # 11zestimate last updated
        try:    
            zestimate_last_updated = result.zestimate_last_updated
        except AttributeError:
            home_type = None
        # 12zestimate value change
        try:    
            zestimate_value_change = result.zestimate_value_change
        except AttributeError:
            zestimate_value_change = None
        
        final = [zillow_id,tax,tax_year,home_type,
                 property_size,bathrooms,bedrooms,
                 last_sold_date,last_sold_price,zestimate_amount,
                 zestimate_last_updated,zestimate_value_change]
        
    except:
        final = []
    return final

In [10]:
get_zillow_info(bad_address,bad_zip)

[]

In [11]:
address_zip_df["zillow_info"] = address_zip_df.progress_apply(lambda row: get_zillow_info(row["Address"],row["Zip"]),axis=1)

21241it [1:16:22,  7.34it/s]


In [12]:
address_zip_df.head()#["zillow_info"][10][10]

,Address,Zip,zillow_info
0,5 14th,60605,"[60204228, 248760.0, 2017, Condominium, 10593,..."
1,8 9th,60605,"[119088436, 40600.0, 2017, Miscellaneous, 4312..."
2,234 Ashland,60605,[]
3,1733 Congress,60605,[]
4,1753 Congress,60605,[]


In [13]:
# turn the zillow_info column which is a list into dataframe
zillow_info_df = pd.DataFrame(address_zip_df.zillow_info.values.tolist())

In [14]:
zillow_info_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,60204228,248760.0,2017,Condominium,10593,2.0,2,05/27/2004,295000,395223,04/03/2019,48923
1,119088436,40600.0,2017,Miscellaneous,4312440,None,0,03/09/2018,300000,None,01/01/1970,None
2,None,None,None,None,None,None,None,None,None,None,None,None
3,None,None,None,None,None,None,None,None,None,None,None,None
4,None,None,None,None,None,None,None,None,None,None,None,None


In [15]:
# CHANGE COLUMN NAME
# zillow_info_df = zillow_info_df.rename(columns={0:"Zillow ID", 1:"Tax"})
zillow_info_df = zillow_info_df.rename(columns={0:"Zillow ID",1:"Tax",2:"Tax_Year",3:"Home Type",4:"Home Size",5:"Bathrooms",6:"Bedrooms",7:"Last Sold Date",8:"Last Sold Price",9: "Zestimate Amount",10:"Zestimate Date",11:"Zestimate Value Change"})
#df = df.rename(columns={'oldName1': 'newName1', 'oldName2': 'newName2'})
zillow_info_df

,Zillow ID,Tax,Tax_Year,Home Type,Home Size,Bathrooms,Bedrooms,Last Sold Date,Last Sold Price,Zestimate Amount,Zestimate Date,Zestimate Value Change
0,60204228,248760.0,2017,Condominium,10593,2.0,2,05/27/2004,295000,395223,04/03/2019,48923
1,119088436,40600.0,2017,Miscellaneous,4312440,None,0,03/09/2018,300000,None,01/01/1970,None
2,None,None,None,None,None,None,None,None,None,None,None,None
3,None,None,None,None,None,None,None,None,None,None,None,None
4,None,None,None,None,None,None,None,None,None,None,None,None
5,2093316676,None,None,Apartment,None,1.0,1,02/01/2017,2415,None,01/01/1970,None
6,2095670559,None,None,Apartment,None,1.0,1,11/10/2016,2500,None,01/01/1970,None
7,None,None,None,None,None,None,None,None,None,None,None,None
8,70474983,513000.0,2017,Condominium,78844,2.0,2,07/20/2012,525000,657483,04/03/2019,-597
9,70474811,490400.0,2017,Condominium,23332,2.0,2,04/17/2013,565000,645588,04/03/2019,-24410


In [16]:
# combine zillow info dataframe with address info, to make sure they line up
final_zillow = pd.concat([address_zip_df,
                          zillow_info_df],axis=1)

In [17]:
# check results
final_zillow.head()

,Address,Zip,zillow_info,Zillow ID,Tax,Tax_Year,Home Type,Home Size,Bathrooms,Bedrooms,Last Sold Date,Last Sold Price,Zestimate Amount,Zestimate Date,Zestimate Value Change
0,5 14th,60605,"[60204228, 248760.0, 2017, Condominium, 10593,...",60204228,248760.0,2017,Condominium,10593,2.0,2,05/27/2004,295000,395223,04/03/2019,48923
1,8 9th,60605,"[119088436, 40600.0, 2017, Miscellaneous, 4312...",119088436,40600.0,2017,Miscellaneous,4312440,None,0,03/09/2018,300000,None,01/01/1970,None
2,234 Ashland,60605,[],None,None,None,None,None,None,None,None,None,None,None,None
3,1733 Congress,60605,[],None,None,None,None,None,None,None,None,None,None,None,None
4,1753 Congress,60605,[],None,None,None,None,None,None,None,None,None,None,None,None


In [18]:
# check file path, just like `pwd` in terminal 
import os
os.getcwd()

'/Users/miiinaa223/Desktop/Python/NW-Bootcamp-Projects'

In [19]:
# drop zillow_info column AND save file
final_zillow.drop(columns=["zillow_info"]).to_csv("final_zillow_info.csv",index=None)